<a href="https://colab.research.google.com/github/krystofaser/personal_projects/blob/main/COT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import re
import plotly.graph_objects as go
from plotly import subplots
from sklearn.preprocessing import MinMaxScaler

In [ ]:
API_key = '941831646d59288f153aa1fd9ee664f1'
report = 'CT'

def get_base_future_name(text):
  text = re.sub(r'\([^)]*\)', '', text)  # Remove bracketed content
  text = re.sub(r's(?!\w)', '', text) # remove S that is not followed by a word
  text = text.rstrip()   # Remove trailing whitespace
  return text

In [ ]:
# extract raw data
url = f'https://financialmodelingprep.com/api/v4/commitment_of_traders_report/{report}?apikey={API_key}'
raw_data = requests.get(url).json()
#convert raw data to df
raw_df = pd.DataFrame(raw_data)[['short_name', 'market_and_exchange_names','date','comm_positions_long_all','comm_positions_short_all','noncomm_positions_long_all','noncomm_positions_short_all','nonrept_positions_long_all','nonrept_positions_short_all', 'open_interest_all']]
raw_df.columns = ['short_name', 'market_and_exchange_names','date', 'comm_long', 'comm_short', 'large_specs_long', 'large_specs_short', 'small_specs_long', 'small_specs_short', 'open_interest']

#Get name of the future examined from raw df
market_and_exchange = raw_df['market_and_exchange_names'][0]
commodity_name = get_base_future_name(raw_df['short_name'][0])

#create the actual df containing only data i will use
df = pd.DataFrame(raw_df[['date', 'open_interest']])
#convert short & long positions into net positions
df['commercials'] = raw_df['comm_long'] - raw_df['comm_short']
df['large_speculators'] = raw_df['large_specs_long'] - raw_df['large_specs_short']
df['small_speculators'] = raw_df['small_specs_long'] - raw_df['small_specs_short']

#convert date column to index of df with datetime format
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.index.name = None

#GET FUTURE DATA

#Get the name of the future
future_name = f"{commodity_name} Futures"

#Get the symbol of the future and the currency it is reported in from the futures list available on financial modeling prep
url = f'https://financialmodelingprep.com/api/v3/symbol/available-commodities?apikey={API_key}'
future_list_df = pd.DataFrame(requests.get(url).json())
try:
  symbol = future_list_df.loc[future_list_df['name'] == future_name, 'symbol'].values[0]
  currency = future_list_df.loc[future_list_df['name'] == future_name, 'currency'].values[0]
except:
  try:
    symbol = future_list_df.loc[future_list_df['name'] == commodity_name, 'symbol'].values[0]
    currency = future_list_df.loc[future_list_df['name'] == commodity_name, 'currency'].values[0]
  except:
    print("Future not found")

#Get the futures historical data

url = f'https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}?apikey={API_key}'
raw_futures = requests.get(url).json()
futures_df = pd.DataFrame(raw_futures['historical'])
futures_df = futures_df[['date', 'open', 'high', 'low', 'close']]

futures_df['date'] = pd.to_datetime(futures_df['date'])
futures_df = futures_df.set_index('date')
futures_df.index.name = None


df.head(-1)

Future not found


NameError: name 'symbol' is not defined

In [ ]:
fig = subplots.make_subplots(rows = 3, cols = 1, subplot_titles = ('COT - traders', 'Open Interest', f'{future_name} - {currency}'), row_heights = [0.5,0.1,0.4])

fig.update_layout(
    title= market_and_exchange,
    xaxis_title='Date',
    yaxis_title='Value',
    height = 1000,
    xaxis_range = (futures_df.index[-1],futures_df.index[0])
)

fig.update_yaxes(
    range = (df[['large_speculators', 'commercials', 'small_speculators']].min(), df[['large_speculators', 'commercials', 'small_speculators']].max()),
    row = 2,
    col = 1
)

fig.add_trace(go.Bar(x = df.index, y = df['commercials'], name = 'Commercial', marker_color = 'red', width = 1000000000/2), row = 1, col = 1)
fig.add_trace(go.Bar(x = df.index, y = df['large_speculators'], name = 'Large Speculators', marker_color = 'blue', width = 1000000000/2), row =1, col = 1)
fig.add_trace(go.Bar(x = df.index, y = df['small_speculators'], name = 'Small Speculators', marker_color = 'yellow', width = 1000000000/2), row =1, col = 1)
fig.add_trace(go.Scatter(x = df.index, y = df['open_interest'], name = 'Open Interest', marker_color = 'green'), row = 2, col = 1)
fig.add_trace(go.Scatter(x = futures_df.index, y = futures_df['close'], name = 'Futures', marker_color = 'black'), row = 3, col = 1)

fig.show()